In [46]:
from data import sample_data

import numpy as np
import xgboost as xgb
from scipy.stats import binom_test

In [19]:
DATA = sample_data.uci.uci_heart_numpy()


def sample_numpy_arr(arr: np.ndarray, n: int, seed: int = 0) -> np.ndarray:
    perm = np.random.RandomState(seed).permutation(len(arr))[:n]
    return arr[perm]


def split(x):
    l = len(x) // 2
    return x[:l], x[l:]


def to_domain_dmatrix(x, y):
    assert len(x) == len(y)
    return xgb.DMatrix(np.concatenate([x, y]), label=np.concatenate([np.zeros(len(x)), np.ones(len(y))]))

In [8]:
p_all, q_all = DATA['iid_test_data'], DATA['ood_test_data']

In [49]:
from tqdm import tqdm

N = 10
res = []
for N in range(10, 101, 10):
    res.append([])
    for seed in tqdm(range(100)):
        p1, p2 = split(sample_numpy_arr(p_all, N, seed=seed))
        q1, q2 = split(sample_numpy_arr(q_all, N, seed=seed))
        d1 = to_domain_dmatrix(p1, q1)
        d2 = to_domain_dmatrix(p2, q2)

        PARAMS = {
            'objective': 'multi:softprob',
            'num_class': 2,
            'eval_metric': 'merror',
            'eta': 0.1,
            'max_depth': 6,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'min_child_weight': 1,
            'nthread': 4,
            'tree_method': 'gpu_hist',
            'seed': seed
        }

        bst = xgb.train(PARAMS, d1, num_boost_round=10)
        res[-1].append(binom_test(x=float(bst.eval(d2).split(':')[1]) * N, n=N, p=0.5) <= 0.05)
res = np.array(res)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.52it/s]


In [60]:
data = []
for i, N in enumerate(range(10, 101, 10)):
    x = [res[i].mean(), res[i].std() / np.sqrt(100)]
    if N in (10, 20, 50):
        print(f'${x[0]:.2f} \pm {x[1]:.2f}$'.replace('0.', '.'), end=' & ')
    data.append((N, x[0]))

$.15 \pm .04$ & $.51 \pm .05$ & $.98 \pm .01$ & 

0.34375